In [1]:
import requests
from bs4 import BeautifulSoup


In [ ]:
url= 'https://www.anytimemailbox.com/'

# Request
r1 = requests.get(url)
#Check response status if 200 then its ok
r1.status_code

# We'll save in coverpage the cover page content
main_page = r1.content

# Soup creation
soup1 = BeautifulSoup(main_page, 'html5lib')
soup1

In [14]:
link_class=soup1.find_all('a',class_='btn theme-button')
for links in link_class:
    if links['href']=='/locations':
        loc_link=links.get('href')
        print(links.get('href'))
        break
location_link=url+loc_link
print(location_link)

/locations
https://www.anytimemailbox.com//locations


In [20]:
from tqdm import tqdm
import os

In [23]:
country=[]
state=[]
title=[]
address=[]
price=[]

In [24]:
#get locations
r2 = requests.get(location_link)
r2.status_code

# We'll save in coverpage the cover page content
location_page = r2.content

# Soup creation
soup2 = BeautifulSoup(location_page, 'html5lib')

# Search countries
country_url = soup2.find_all('div', class_="row theme-top-gap")
count=0
for countr in tqdm(country_url):
    # Search country URL
    tmp=(countr.find('a')['href'])
    country_name=os.path.basename(tmp)
    country_url_full=url+tmp[1:]
    #get country page
    r3 = requests.get(country_url_full)
    r3.status_code

    # country page content
    country_wise = r3.content

    # Soup creation
    soup3 = BeautifulSoup(country_wise, 'html5lib')
    # find cites
    checking_cities = soup3.find_all('a', class_="theme-subtle-link")
    for stat in checking_cities:

        state_name=os.path.basename(stat.get('href'))
        #city URL
        stat_loc=country_url_full+'/'+state_name
        #GET city locations
        r4 = requests.get(stat_loc)
        r4.status_code
        # city location content
        coverpage = r4.content
        soup4 = BeautifulSoup(coverpage, 'html5lib')
        #locations details
        adrr_loc = soup4.find_all('div', class_="theme-location-item")
        for event in adrr_loc:
            # country data
            country.append(country_name)
            # city data
            state.append(state_name)
            #Address Title data
            title_address=event.find('h3')
            title.append(title_address.text)
            #Price data
            price_list=event.find('div',class_='t-price')
            price_list=price_list.text.splitlines()[3].replace('\t','')
            price.append(price_list)
            ## Full Address
            addres_list=event.find('div',class_='t-addr').get_text(separator=" ").strip()
            address.append(addres_list)


100%|██████████| 42/42 [01:14<00:00,  1.78s/it]


In [25]:
import pandas as pd

In [26]:
data = pd.DataFrame(
          {'Country':country,
            'State':state,
            'Address Title': title,
           'Address':address, 
           'Price':price,
        })

In [27]:
data

,Country,State,Address Title,Address,Price
0,usa,alabama,Birmingham - 19th St,"120 19th Street North Birmingham, AL 35203",US$ 39.00 / month
1,usa,alabama,Birmingham - Gadsden Hwy,"1430 Gadsden Highway Birmingham, AL 35235",US$ 9.99 / month
2,usa,alabama,Chelsea,"60 Chelsea Corners Chelsea, AL 35043",US$ 9.99 / month
3,usa,alabama,Enterprise,"2801 Rucker Blvd Enterprise, AL 36330",US$ 9.99 / month
4,usa,alabama,Mobile - Grelot Rd,"6300 Grelot Road Mobile, AL 36609",US$ 9.99 / month
...,...,...,...,...,...
1339,united-kingdom,london,London - Ludgate Hill,39 Ludgate Hill London EC4M 7JN United Kingdom,£5.99 / month
1340,united-kingdom,london,London - New North Rd,275 New North Road London N1 7AA United Kingdom,£9.99 / month
1341,united-kingdom,london,London - Regent St,"59, St. Martin's Lane London WC2N 4JS United K...",£17.99 / month
1342,united-kingdom,hampshire,Southampton,Millview Barn Southampton SO31 8GD United Kingdom,£10.00 / month
